In [ ]:
from pathlib import Path
from bertopic import BERTopic
import pandas as pd
import re
import numpy as np
import math
from umap import UMAP
from collections import Counter
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
from sklearn.feature_extraction.text import CountVectorizer


# Load and wrangle the data

In [ ]:
# read the data
src = "../../data/tweets"
fname = "combined_US_politician_twitter_timelines_2010-11-06_to_2022-03-16_lemma.csv.gzip"
tweets = pd.read_csv(
    Path(src, fname),
    compression="gzip",
    dtype={"id":str}
)
tweets.head()

In [ ]:
# clean the data and creating a list of docs for the model
tweets['lemmatized'] = tweets['lemmatized'].replace(r'\s+|\\n', ' ', regex=True)

In [ ]:
docs = tweets.lemmatized.to_list()
docs = [item for item in docs if str(item) != 'nan']
filtered_data = data.dropna(subset=["lemmatized"])
len(filtered_data) == len(docs)

# Fit the model

In [ ]:
# increase `n_neigbors` and/or `min_topic_size` if too many topics:
umap_model = UMAP(
    n_neighbors=150, 
    n_components=5, 
    metric="cosine", 
    low_memory=False
)

vectorizer_model = CountVectorizer(min_df=50)

topic_model = BERTopic(
    verbose=True,
    nr_topics="auto",
    min_topic_size=200,
    umap_model=umap_model,
    top_n_words=10,
    vectorizer_model=vectorizer_model,
    language="english"
)
topics, probs = topic_model.fit_transform(docs)

In [ ]:
# save the model
dst = "../../data/tweets"
fname = "BERTopic_model"
topic_model.save(Path(dst, fname))

# Export topic data

In [ ]:
# create a table with docs and respective topics
df_all_docs = pd.DataFrame({
    "id": filtered_data["id"], 
    'topic': topics
})

# add belief-speaking and truth-seeking similarity
src = "../../data/tweets"
fname = "US_politician_tweets_2010-11-06_to_2022-03-16.csv.gzip"
cols = ["id", "avg_belief_score", "avg_truth_score"]
similarities = pd.read_csv(
    Path(src, fname),
    usecols=cols,
    compression="gzip",
    dtype={"id":str}
)
df_all_docs = pd.merge(
    df_all_docs,
    similarities,
    how="left"
)

# save for plotting belief-speaking similarity and truth-seeking
# similarity in figure 1
dst = "../../data/tweets"
fname = "topics_all_docs.csv.gzip"
df_all_docs.to_csv(
    Path(dst, fname),
    compression="gzip",
    index=False
)

# Inspect the model

In [ ]:
src = "../../data/tweets"
fname = "BERTopic_model"
topic_model = BERTopic.load(Path(src, fname))

In [ ]:
# check how many topics the model found
dst = "../../data/tweets"
fname = "topics_info.csv"
topic_info = pd.DataFrame(topic_model.get_topic_info())
topic_info.to_csv(Path(dst, fname), index=False)

In [ ]:
# visualize the topics (call `x.write_html(path)` to save the graphs)
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_heatmap(n_clusters=20)

In [ ]:
# Dynamic Topic Modelling (topics over time):
topics, probs = topic_model.transform(docs)
timestamps = data.created_at.to_list()
topics_over_time = topic_model.topics_over_time(docs, topics, timestamps, nr_bins=20)

In [ ]:
# creating topics per class dataframe to check for topic distribution over 
# party/components
topics_per_class = topic_model.topics_per_class(
    docs=docs, 
    topics=topics, 
    classes=filtered_data["classes_quant"], 
    global_tuning=True
)

In [ ]:
topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=100)

In [ ]:
# save topic names for plotting in figure 1
dst = "../../data"
fname = "topics_per_class_ddr.csv"
topics_per_class_named.to_csv(Path(dst, fname), index=False)

# Prepare data to create the scattertext plot

In [ ]:
topics_ids = []  # insert topic id(s)
topics_words = []
topic_embeddings = []

In [ ]:
for id in topics_ids:
    word_list = []
    embedding_list = []
    for i in range(
        0, 20
    ):  # insert how many words you want to display (=< top_n_words in model computation)
        word = topic_model.get_topic(id)[i][0]
        word_list.append(word)
        embedding = topic_model.get_topic(id)[i][1]
        embedding_list.append(embedding)
    topics_words.append(word_list)
    topic_embeddings.append(embedding_list)


In [ ]:
wordcloud_df = pd.DataFrame()
wordcloud_df["topic_ids"] = topics_ids
wordcloud_df["topic_words"] = topics_words
wordcloud_df["topic_embeddings"] = topic_embeddings

In [ ]:
dst = "../../tweets"
fname = "key_topics.csv"
wordcloud_df = wordcloud_df.set_index(["topic_ids"]).apply(pd.Series.explode).reset_index()
wordcloud_df.to_csv(Path(dst, fname))